In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [36]:
cp kaggle.json /root/.config/kaggle

In [38]:
!chmod 600 /root/.config/kaggle/kaggle.json

In [39]:
cp kaggle.json /root/.config/kaggle

In [40]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogsvscats")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'dogsvscats' dataset.
Path to dataset files: /kaggle/input/dogsvscats


In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import pickle

In [44]:
batch_size = 32


train_ds = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,

)

test_ds = ImageDataGenerator(rescale=1./255)


train_generator = train_ds.flow_from_directory(
    '/kaggle/input/dogsvscats/train',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'
)

validation_generator = test_ds.flow_from_directory(
    '/kaggle/input/dogsvscats/test',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [45]:
#  create CNN model\

model = Sequential()

model.add(Conv2D(32, (3, 3),padding='valid', activation = 'relu',input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

model.add(Conv2D(64, (3, 3),padding='valid', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))


model.add(Conv2D(128, (3, 3),padding='valid', activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
model.fit(
    train_generator,
    steps_per_epoch=10000 // batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=2000 // batch_size
)

Epoch 1/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 193s 618ms/step - accuracy: 0.6025 - loss: 0.7451 - val_accuracy: 0.6789 - val_loss: 0.6004
Epoch 2/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 175s 562ms/step - accuracy: 0.6833 - loss: 0.6002 - val_accuracy: 0.7167 - val_loss: 0.5539
Epoch 3/5
  1/312 ━━━━━━━━━━━━━━━━━━━━ 30s 99ms/step - accuracy: 0.6562 - loss: 0.5686

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


312/312 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6562 - loss: 0.5686 - val_accuracy: 0.7203 - val_loss: 0.5439
Epoch 4/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 166s 532ms/step - accuracy: 0.7157 - loss: 0.5623 - val_accuracy: 0.6860 - val_loss: 0.5891
Epoch 5/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 163s 524ms/step - accuracy: 0.7419 - loss: 0.5255 - val_accuracy: 0.7465 - val_loss: 0.5460


In [48]:
print(train_generator.samples)        # total number of images
print(train_generator.batch_size)     # batch size used
print(train_generator.n)              # same as .samples
print(train_generator.class_indices)  # mapping of class names to integers


20000
32
20000
{'cats': 0, 'dogs': 1}
